# HKJC Crawler Script

- To monitor odd trend (specifically Corner HiLow), only for in play betting 

According to https://medium.com/@chris_whirlwind/web-scraping-football-match-odds-on-hk-jockey-club-7c605d0ab038

In [1]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver

## Test connection

 - Download Chrome driver from https://sites.google.com/a/chromium.org/chromedriver/downloads
 - Put the driver executable file into the working directory

In [11]:
driver = webdriver.Chrome('/Users/TysonWu/dev/odds-crawl/odds-crawl/chromedriver')
driver.get('http://www.google.com/');
time.sleep(5) # Let the user actually see something!
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()
time.sleep(5) # Let the user actually see something!
driver.quit()

- Natvigate to bet url and copy the address, paste it onto input url
- type the date, weekday, number

In [2]:
#---------input-url---
url = 'https://bet.hkjc.com/football/odds/odds_allodds.aspx?lang=EN&tmatchid=8f455de2-e486-4c17-945d-44c4bf942d01'
date = '20200212' #YYYYmmdd format
weekday = 'SUN' # capitalized letter please
number = '32' # event number
game_starting_time = '20200210 03:45' #YYYYmmdd HH:MM format
#--------------------
event_id = date+weekday+number

### Reference

get div according to div id and div class by selenium:

https://stackoverflow.com/questions/36615472/how-to-find-element-in-selenium-python-using-id-and-class-in-div
https://stackoverflow.com/questions/29807856/selenium-python-how-to-get-texthtml-source-from-div

get content inside span, remove tags:

https://stackoverflow.com/questions/42175190/get-value-of-span-tag-using-beautifulsoup/42175359

In [50]:
import sys
div_id = 'dCHLTable'+event_id

driver = webdriver.Chrome('/Users/TysonWu/dev/odds-crawl/development/chromedriver')
driver.get(url)
time.sleep(5)
# html_source = driver.page_source # get WHOLE html source which wastes time
try:
    chlodds = driver.find_element_by_xpath("//div[@id='"+div_id+"'"+"and @class='betTypeAllOdds']")
    chlodds_content = chlodds.get_attribute('innerHTML')
except:
    driver.quit()
    print('Error finding attribute. This is mostly caused by an end of game or incorrect input. Program will now exit.')
    sys.exit()
finally:
    driver.quit()

Error finding attribute. This is mostly caused by an end of game or incorrect input. Program will now exit.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-50-0fe3986a2f2f>", line 9, in <module>
    chlodds = driver.find_element_by_xpath("//div[@id='"+div_id+"'"+"and @class='betTypeAllOdds']")
  File "/opt/anaconda3/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 394, in find_element_by_xpath
    return self.find_element(by=By.XPATH, value=xpath)
  File "/opt/anaconda3/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 978, in find_element
    'value': value})['value']
  File "/opt/anaconda3/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "/opt/anaconda3/lib/python3.7/site-packages/selenium/webdriver/remote/errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/

SystemExit: 

### Alternative: Without actually opening the driver (use HTMLUnit)

https://stackoverflow.com/questions/4081724/how-do-i-use-the-htmlunit-driver-with-selenium-through-the-python-bindings

- Install Selenium webdriver:

https://pypi.org/project/selenium/

Whole content: dMixContent_20200209SUN24


Corner HiLo: \<div id="dCHLTable20200209SUN27" class="betTypeAllOdds"\>

In [98]:
def get_odds(event_id):
    entry = '1'
    line_list = []
    chl_hi_list = []
    chl_low_list = []
    if soup.find('span', id = event_id+"_CHL_LINE_"+entry).text == '---': # suspend time
        return [],[],[]
    while (soup.find('span', id = event_id+"_CHL_LINE_"+entry)): # exit when return None
        # get text content from a specific span id
        line = soup.find('span', id = event_id+"_CHL_LINE_"+entry).text.strip('[]')
        chl_hi = soup.find('span', id = event_id+"_CHL_H_"+entry).text
        chl_low = soup.find('span', id = event_id+"_CHL_L_"+entry).text
        entry = str(int(entry)+1)
        line_list.append(line)
        chl_hi_list.append(chl_hi)
        chl_low_list.append(chl_low)
    return line_list, chl_hi_list, chl_low_list

# pretty by beautiful soup
soup = BeautifulSoup(chlodds_content, 'html.parser')
# print(soup.prettify())
line_list, chl_hi_list, chl_low_list = get_odds(event_id)

Now we got a list of odds in this instance.

Turn the result into a pandas dataframe:

In [130]:
import pandas as pd
import os
import os.path
import time
from datetime import datetime

timestamp = datetime.now()
timestamp = datetime.strftime(timestamp, '%Y-%m-%d %H:%M:%S')
timestamp = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')

odds = pd.DataFrame({'timestamp': timestamp, 
                     'line':line_list, 
                     'corner_hi':chl_hi_list,
                     'corner_low': chl_low_list})
odds['game_starting_time'] = game_starting_time
odds['game_starting_time'] = odds['game_starting_time'].apply(lambda x: datetime.strptime(x, '%Y%m%d %H:%M'))
odds['minutes'] = odds.timestamp - odds.game_starting_time
odds['minutes'] = odds['minutes'].apply(lambda x: datetime.strptime(x, '0 days %H:%M:%S.%f000').strftime('%H:%M:%S'))

odds

,timestamp,line,corner_hi,corner_low,game_starting_time,minutes
0,2020-02-10 04:37:19.713333,10.5,1.74,1.97,2020-02-10 03:45:00,00:52:19.713333
1,2020-02-10 04:37:19.713333,11.5,2.17,1.61,2020-02-10 03:45:00,00:52:19.713333


In [ ]:
# if file exists, write line; if file does not exist, create one and write line
path_file = os.getcwd()+'/data/'+event_id+'.csv'
if os.path.isfile(path_file):
    odds.to_csv(path_file, index=False, mode='a', header=False)
else:
    odds.to_csv(path_file, index=False, mode='w', header=True)

### Test: these damn times

In [33]:
from datetime import datetime
timestamp = datetime.now()
timestamp = datetime.strftime(timestamp, '%Y-%m-%d %H:%M:%S') # convert datetime to str
timestamp = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S') # convert str to datetime
print(timestamp)

timestamp2 = '20200210 03:45' # str
timestamp2 = datetime.strptime(timestamp2, '%Y%m%d %H:%M') # convert str to datetime
print(timestamp2)

diff = timestamp - timestamp2 # type: timedelta

2020-02-10 07:44:08
2020-02-10 03:45:00


# Automation

https://www.youtube.com/watch?v=QZJ1drMQz1A

### Check Python version

- Can be done by typing `which python` in Terminal

1. Type `crontab -e` in terminal to enter vim edit cron jobs
2. Press `I` to go to insert mode in Vim
3. Specify the time for job, then insert Python path got from `which python`, then insert crawler.py path
4. Press `Esc` after finishing input
5. Type `:wq` to save and quit

#### Time Format

```
# ┌───────────── minute (0 - 59)
# │ ┌───────────── hour (0 - 23)
# │ │ ┌───────────── day of month (1 - 31)
# │ │ │ ┌───────────── month (1 - 12)
# │ │ │ │ ┌───────────── day of week (0 - 6) (Sunday to Saturday;
# │ │ │ │ │                                       7 is also Sunday on some systems)
# │ │ │ │ │
# │ │ │ │ │
# * * * * *  command_to_execute
```

#### Cron job utilities
`crontab -l` shows a list of cronjobs

#### Example

```
* 8-11 * * * /opt/anaconda3/bin/python /Users/TysonWu/dev/hkjc-crawl/crawler.py
* 8-11 * * * ( sleep 30 ; /opt/anaconda3/bin/python /Users/TysonWu/dev/hkjc-crawl/crawler.py )
```
The first line runs script at 8-11am every minute

The second line runs script 30 seconds after the first one -> run the script every 30 seconds

---

# Future development:

1. Use headless drivers instead of chromedriver which opens up every time

2. Consistency on the timestamp

3. Automate the input -> user input url only, automate getting other infos

4. Crawl multiple games simultaneously

5. Flows data into MySQL server


## Auto get game info

https://selenium-python.readthedocs.io/locating-elements.html

In [44]:
#---------input-url---
url = 'https://bet.hkjc.com/football/odds/odds_allodds.aspx?lang=EN&tmatchid=8f455de2-e486-4c17-945d-44c4bf942d01'
game_starting_time = '20200212 18:00' #YYYYmmdd HH:MM format
#--------------------

##### With auto crawl game starting time 

In [24]:
import sys
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
time.sleep(5)
# html_source = driver.page_source # get WHOLE html source which wastes time
try:
    game_info = driver.find_element_by_id('litMDay')
    game_starting_time = driver.find_element_by_id('litEsst')
except:
    driver.quit()
    print('Error finding attribute. Program will now exit.')
    sys.exit()
finally:
    game_info_content = game_info.get_attribute('innerHTML')
    game_starting_time_content = game_starting_time.get_attribute('innerHTML')
    driver.quit()


Looking for [chromedriver 80.0.3987.16 mac64] driver in cache 
File found in cache by path [/Users/TysonWu/.wdm/drivers/chromedriver/80.0.3987.16/mac64/chromedriver]


In [23]:
from datetime import datetime
game_starting_time_content = datetime.strptime(game_starting_time_content, '%d/%m %H:%M').strftime('%H:%M')
if game_starting_time_content >= '18:00':
    print('here')
else:
    print('wrong')

wrong


#### Without auto crawl game starting time 
Only auto crawl weekday, number, date

In [45]:
import sys
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
time.sleep(5)
# html_source = driver.page_source # get WHOLE html source which wastes time
try:
    game_info = driver.find_element_by_id('litMDay')
except:
    driver.quit()
    print('Error finding attribute. Program will now exit.')
    sys.exit()
finally:
    game_info_content = game_info.get_attribute('innerHTML')
    driver.quit()


Looking for [chromedriver 80.0.3987.16 mac64] driver in cache 
File found in cache by path [/Users/TysonWu/.wdm/drivers/chromedriver/80.0.3987.16/mac64/chromedriver]


In [46]:
from datetime import timedelta

game_info_content = game_info_content.replace(' ','')
start_date, start_time = game_starting_time.split()[0], game_starting_time.split()[1]
start_time = datetime.strptime(start_time, '%H:%M').strftime('%H:%M')
start_date = datetime.strptime(start_date, '%Y%m%d')
if start_time <= '12:00':
    start_date = start_date - timedelta(days=1)
start_date = datetime.strftime(start_date, '%Y%m%d')

print(start_date)
#     date = date.strftime()

20200212


In [47]:
event_id = start_date+game_info_content